## Data visualization (and statistics)

Visualization and analysis for the fingerprint experiment (hand sanitizer project DSPH). 

Count data can be found in microsoft teams "transmission and hand sanitizer".
I manually changed the names in the excel file containing the count data.

In an older version mock data is created.

Contains:
- importing data from excel file
- make visualization
    - nested bar graph

- statistical test
    - testing for normality : Anderson-Darling
    - statistics : two-sample t-test


Improvements:
- add error bar   
- create function for the bar graph code
- make function incorporation test for normality and significance testing


In [1]:
# imports

# data manipulation and processing
import pandas as pd
import numpy as np

# visualizations
from bokeh.models import FactorRange, Legend, ColumnDataSource, Whisker, LegendItem
from bokeh.palettes import Colorblind
from bokeh.plotting import figure, show, output_notebook, output_file

# statistics
from scipy.stats import ttest_ind, mannwhitneyu, anderson, shapiro

output_file('results.html')


### import data fingerprint experiment

In [2]:
df = pd.read_excel('fingerprinting.xlsx')
df.set_index('experiment', inplace=True)
df = df.transpose()
df

experiment,HS1-1,HS1-2,HS1-3,HS1-C1,HS1-C2,HS1-C3
1,42,90,90,26,30,90
2,79,120,57,6,28,66
3,3,11,3,7,30,3
4,6,76,118,12,81,30
5,17,22,9,40,5,9
6,57,43,100,62,27,47


In [3]:
# get the mean for each triplicate
df['HS1_S'] = df[df.columns[df.columns.str.contains('HS1-\d+')]].mean(axis=1)
df['HS1_C'] = df[df.columns[df.columns.str.contains('HS1-C\d+')]].mean(axis=1)
#df['HS2_S'] = df[df.columns[df.columns.str.contains('HS2-\d+')]].mean(axis=1)
#df['HS2_C'] = df[df.columns[df.columns.str.contains('HS2-C')]].mean(axis=1)  

df_res = df.iloc[:, -2:].T # change depending on the number of HS (# HS * 2)
df_res['count'] = df_res[df_res.columns].mean(axis=1)

df_res



,1,2,3,4,5,6,count
experiment,,,,,,,
HS1_S,74.000000,85.333333,5.666667,66.666667,16.0,66.666667,52.388889
HS1_C,48.666667,33.333333,13.333333,41.000000,18.0,45.333333,33.277778


In [4]:
df_res.reset_index(inplace=True)
df_res[['HS', 'sample_type']] = df_res['experiment'].str.split('_', n=1, expand=True)
df_res.drop(['experiment'], axis=1, inplace=True)


In [20]:
df_res

,1,2,3,4,5,6,count,HS,sample_type,std_dev,x,error
0,74.000000,85.333333,5.666667,66.666667,16.0,66.666667,52.388889,HS1,treated,36.133702,"(HS1, treated)",36.133702
1,48.666667,33.333333,13.333333,41.000000,18.0,45.333333,33.277778,HS1,control,14.988143,"(HS1, control)",14.988143


In [6]:
df_res['std_dev'] = df_res.iloc[:, :5].std(axis=1)


## visualization

In [7]:
df_res['sample_type'] = df_res['sample_type'].map({'S': 'treated', 'C': 'control'})
df_res['x'] = df_res.apply(lambda row: (row['HS'], str(row['sample_type'])), axis=1)

In [21]:
# nested bar graph: https://stackoverflow.com/questions/67901133/create-nested-bar-graph-in-bokeh-from-a-dataframe
# legend append: https://stackoverflow.com/questions/46730609/position-the-legend-outside-the-plot-area-with-bokeh

df_res.columns = df_res.columns.astype(str)

p = figure(
    x_range=FactorRange(*list(df_res["x"].unique())),
    width=500
)

factors = df_res['sample_type'].unique()

# Manually specify colors for each factor
colors = Colorblind[3][:len(factors)]

legend_items = []

for i, factor in enumerate(factors):
    source = ColumnDataSource(df_res[df_res['sample_type'] == factor])
    
    vbar = p.vbar(x='x', top='count', width=0.9, source=source,
                  color=colors[i])
    legend_items.append((factor, [vbar]))


p.y_range.start = 0
p.y_range.end = df_res['count'].max() * 1.2
p.x_range.range_padding = 0.25

p.title = "Number of colonies per handsanitizer"
p.title.text_font_size = '15px'
p.yaxis.axis_label = "Number of colonies"
p.xaxis.axis_label = "Hand Sanitizers"
p.xgrid.grid_line_color = None

# Create a legend
legend = Legend(items=legend_items, location="top_center")
legend.label_text_font_size = "12px"
legend.spacing = 5
legend.click_policy = "hide"  

p.add_layout(legend, 'below')

# Show the plot
show(p)



In [73]:
# prepare for statistics
# Create a new column by combining 'HS' and 'sample_type'
df_res['HS_sample_type'] = df_res['HS'] + '_' + df_res['sample_type']

df_res.set_index('HS_sample_type', inplace=True)
df_transposed = df_res.T
df_transposed.reset_index(inplace=True)

# Convert 'HS1_treated' and 'HS1_control' to numeric
df_transposed['HS1_treated'] = pd.to_numeric(df_transposed['HS1_treated'][:6])
df_transposed['HS1_control'] = pd.to_numeric(df_transposed['HS1_control'][:6])

treated = df_transposed['HS1_treated'].iloc[:6]
control = df_transposed['HS1_control'].iloc[:6]

In [74]:
print(f'control : {anderson(control)}')
print(f'control : {anderson(treated)}')


control : AndersonResult(statistic=0.3234850193985688, critical_values=array([0.592, 0.675, 0.809, 0.944, 1.123]), significance_level=array([15. , 10. ,  5. ,  2.5,  1. ]), fit_result=  params: FitParams(loc=33.27777777777778, scale=14.649105648342367)
 success: True
 message: '`anderson` successfully fit the distribution to the data.')
control : AndersonResult(statistic=0.5503951623647731, critical_values=array([0.592, 0.675, 0.809, 0.944, 1.123]), significance_level=array([15. , 10. ,  5. ,  2.5,  1. ]), fit_result=  params: FitParams(loc=52.388888888888886, scale=33.067215497237456)
 success: True
 message: '`anderson` successfully fit the distribution to the data.')


In [75]:
# Check normality using Shapiro-Wilk test
stat_treated, p_value_treated = shapiro(treated)
stat_control, p_value_control = shapiro(control)

# Print the results
print(f'Shapiro-Wilk test for normality - HS1 treated: Statistic = {stat_treated}, P-value = {p_value_treated}')
print(f'Shapiro-Wilk test for normality - HS1 control: Statistic = {stat_control}, P-value = {p_value_control}')


Shapiro-Wilk test for normality - HS1 treated: Statistic = 0.8334945440292358, P-value = 0.11504392325878143
Shapiro-Wilk test for normality - HS1 control: Statistic = 0.899553120136261, P-value = 0.37129536271095276


**conclusion** 

The p-value is above 0.05 thus the data is normally distributed.

for the anderson-darling test - for significance level of 0.05: statistic is below the critical value

Thus the H0 is not rejected

In [76]:
# Perform two-sample t-test
t_statistic, p_value = ttest_ind(treated, control)

# Print the results
print(f'Two-sample t-test results for HS1: T-statistic = {t_statistic}, P-value = {p_value}')

Two-sample t-test results for HS1: T-statistic = 1.2943494456999944, P-value = 0.2246352596223862


In [77]:
# Perform Mann-Whitney U test
u_statistic, p_value = mannwhitneyu(treated, control)

# Print the results
print(f'Mann-Whitney U test results for HS1: U-statistic = {u_statistic}, P-value = {p_value}')

Mann-Whitney U test results for HS1: U-statistic = 25.0, P-value = 0.29710698354709286


### conclusion